8기_최준혁

# 필요 라이브러리 호출하기

pandas, numpy 등 데이터 읽기에 필요한 라이브러리를 호출해줍니다. 

In [1]:
# 필요 라이브러리 호출
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

## 파일 읽기

pandas의 read_csv를 활용하여 csv 파일을 데이터프레임 형태로 불러옵니다.

In [2]:
# 파일 불러오기
data = pd.read_csv("./FIFA.csv", index_col = None)
df = data.copy()

In [3]:
# 로지스틱 회귀를 위한 전처리
df.drop(['id','name','continent', 'contract_until'], axis = 1, inplace = True)
df.replace({'prefer_foot' :'?'}, 'right', inplace = True)
df.fillna({'stat_potential':df['stat_overall']}, inplace=True)
df.fillna({'stat_overall':df['stat_potential']}, inplace=True)
df.dropna(how = 'any', inplace = True)

q1 = 375000      # 1분위수
q3 = 2600000     # 3분위수

df = df[(df['value'] > (q1-1.5*(q3-q1))) & (df['value'] < (q3+1.5*(q3-q1)))]
df.reset_index(drop = True, inplace = True)

df = pd.get_dummies(df, 
                    columns = ['position', 'prefer_foot'],
                    drop_first = False)

In [4]:
df.head()

,age,reputation,stat_overall,stat_potential,stat_skill_moves,value,position_DF,position_GK,position_MF,position_ST,prefer_foot_left,prefer_foot_right
0,27,1,63.0,63.0,3,300000,1,0,0,0,0,1
1,32,1,67.0,67.0,1,450000,0,1,0,0,0,1
2,23,1,53.0,60.0,2,70000,1,0,0,0,0,1
3,29,1,67.0,67.0,2,750000,0,0,0,1,0,1
4,23,1,58.0,64.0,3,150000,0,0,1,0,0,1


# 로지스틱 회귀 / 소프트맥스

In [5]:
lr_df_y = df['value']
lr_df_x = df.drop(['value'], axis = 1)

In [6]:
# 종속변수를 연속형 변수에서 범주형 변수로 변환

bins_label = ['value0','value1','value2','value3','value4','value5','value6']
lr_df_y = pd.cut(lr_df_y, 7, right=False, labels=bins_label)

In [7]:
# 종속변수 값 분포 확인

lr_df_y.value_counts()

value0    4349
value1    1630
value2     524
value3     441
value4     329
value5     238
value6     224
Name: value, dtype: int64

# 과제

**과제 1: train_test_split을 이용해 train 데이터와 test 데이터로 분리해보세요!**

In [8]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

X = lr_df_x
y = lr_df_y

train_X, test_X, train_y, test_y = train_test_split(X, y,test_size=0.3, random_state=156)

**과제 2: 로지스틱 회귀 모형을 생성하고, 주어진 param_value를 이용해서 gridSearch로 최적 하이퍼 파라미터를 구해보세요!**

In [9]:
from sklearn.model_selection import GridSearchCV

param_value = {'C':[0.01,0.1,1,10,100]}

# 로지스틱 회귀 모형 생성
model = LogisticRegression()

# gridSearch 모형 생성
gridSearch = GridSearchCV(model,param_grid = param_value,cv=5) 

# gridSearch 모형 학습
gridSearch.fit(train_X,train_y)

GridSearchCV(cv=5, estimator=LogisticRegression(),
             param_grid={'C': [0.01, 0.1, 1, 10, 100]})

**과제3: 로지스틱 회귀 모형의 최적 파라미터 값을 찾아보세요!!**

In [10]:
# gridSearch.best_params를 통해 최적의 하이퍼 파라미터 값을 찾을 수 있습니다

print(gridSearch.best_params_)

{'C': 100}


**과제 4: 최적 하이퍼 파라미터를 통해 로지스틱 회귀 모형을 학습시키고, LogisticRegression().score()를 통해 정확도를 확인해보세요**

In [11]:
# GridSearch를 통해 찾은 하이퍼 파라미터를 지정해 로지스틱 회귀 모형을 생성합니다
lr_clf = LogisticRegression(C = 100)  

# 로지스틱 회귀모형 학습
lr_clf.fit(train_X, train_y)

# 로지스틱 회귀 모형 예측
lr_preds = lr_clf.predict(test_X)

In [12]:
# 로지스틱 회귀 모형 결과확인

# 로지스틱 회귀 모형 학습용 데이터 점수 확인
print(lr_clf.score(train_X, train_y))

# 로지스틱 회귀 모형 검증용 데이터 점수 확인
print(lr_clf.score(test_X, test_y))
'''
원래 결과
0.5759351835890364
0.5791106514994829
''' 

0.5816401920945696
0.5756139595002154


'\n원래 결과\n0.5759351835890364\n0.5791106514994829\n'

질문 : 이거 score는 원래 결과랑 왜 다른건가요? test, train를 어떻게 분리하냐에 따라서 score가 변하긴 하던데 중요하지 않은 문제로 보면 될까요?